In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install numpy matplotlib

Looking in indexes: https://download.pytorch.org/whl/cu121


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
def train(model, device, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

# FGSM Attack
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon*sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def test(model, device, test_loader, epsilon):
    correct_normal = 0  # For normal accuracy
    correct_adv = 0     # For adversarial accuracy
    adv_examples = []

    model.eval()
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        images.requires_grad = True

        # Forward pass the original images
        outputs = model(images)
        _, init_pred = outputs.max(1, keepdim=True)  # Get the index of the max log-probability

        correct_normal += (init_pred == labels.view_as(init_pred)).sum().item()

        # Loop over all examples in batch
        for idx in range(images.size(0)):
            single_image = images[idx].unsqueeze(0)  # add batch dimension
            single_label = labels[idx].unsqueeze(0)  # add batch dimension

            # Forward pass the single image
            output = model(single_image)
            loss = criterion(output, single_label)  # Correct target shape
            model.zero_grad()
            loss.backward()

            # IMPORTANT: Access the gradient after backward and before zero_grad
            data_grad = single_image.grad.data

            # Call FGSM Attack
            perturbed_data = fgsm_attack(single_image, epsilon, data_grad)
            output = model(perturbed_data)
            final_pred = output.max(1, keepdim=True)[1]

            if final_pred.item() == single_label.item():
                correct_adv += 1
            else:
                if len(adv_examples) < 5:  # Save some examples to view later
                    adv_examples.append((init_pred[idx].item(), final_pred.item(), single_image.squeeze().cpu().numpy()))

    # Calculate final accuracy for normal and adversarial examples
    final_acc_normal = correct_normal / float(len(test_loader.dataset))
    final_acc_adv = correct_adv / float(len(test_loader.dataset))
    print(f'Epsilon: {epsilon}\tNormal Test Accuracy = {correct_normal} / {len(test_loader.dataset)} = {final_acc_normal * 100}%')
    print(f'Epsilon: {epsilon}\tAdversarial Test Accuracy = {correct_adv} / {len(test_loader.dataset)} = {final_acc_adv * 100}%')


train(model, device, train_loader, criterion, optimizer)
test(model, device, test_loader, epsilon=0.1)  # Test with epsilon = 0.1


Epoch [1/5], Step [100/938], Loss: 0.26583331823349
Epoch [1/5], Step [200/938], Loss: 0.1979304999113083
Epoch [1/5], Step [300/938], Loss: 0.2215731292963028
Epoch [1/5], Step [400/938], Loss: 0.027187854051589966
Epoch [1/5], Step [500/938], Loss: 0.10458440333604813
Epoch [1/5], Step [600/938], Loss: 0.04334958642721176
Epoch [1/5], Step [700/938], Loss: 0.06950472295284271
Epoch [1/5], Step [800/938], Loss: 0.18899568915367126
Epoch [1/5], Step [900/938], Loss: 0.024309495463967323
Epoch [2/5], Step [100/938], Loss: 0.030632618814706802
Epoch [2/5], Step [200/938], Loss: 0.04329485073685646
Epoch [2/5], Step [300/938], Loss: 0.02735838294029236
Epoch [2/5], Step [400/938], Loss: 0.04314698278903961
Epoch [2/5], Step [500/938], Loss: 0.05688460171222687
Epoch [2/5], Step [600/938], Loss: 0.06867429614067078
Epoch [2/5], Step [700/938], Loss: 0.01020274218171835
Epoch [2/5], Step [800/938], Loss: 0.02424243651330471
Epoch [2/5], Step [900/938], Loss: 0.07050655037164688
Epoch [3/5],

C:\Users\bance\AppData\Local\Temp\ipykernel_30816\2382780966.py:99: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:494.)
  data_grad = single_image.grad.data


AttributeError: 'NoneType' object has no attribute 'data'

: 